In [1]:
# 基本パッケージ（numpy,Pandas,matplotlib）
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# 線形サポートベクターマシーン
from sklearn.svm import LinearSVC
# LightGBM
import lightgbm as lgb
# XGboost
import xgboost as xgb
# train_test_split（データを分割出してくれる）
from sklearn.model_selection import train_test_split
# accuracy_score（正解率を測れる）
from sklearn.metrics import accuracy_score
# グリッドサーチ（ハイパーパラメータを自動的に最適化してくれる）
from sklearn.model_selection import GridSearchCV
# 正規化、標準化用
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
# 特徴量選択用
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE
# 交差検証用
from sklearn.model_selection import cross_val_score
# 多項式特徴量生成用
from sklearn.preprocessing import PolynomialFeatures
# warningの抑制
import warnings
# モデルの保存
from sklearn.externals import joblib

In [2]:
# 表示関連
# DataFrameの列数設定
pd.set_option('display.max_columns', 500)
warnings.filterwarnings('ignore')

In [3]:
# 株価データの読み込み
stock_data = pd.read_csv("stock_Price_Prediction_v3.5.csv", encoding="shift-jis")

# 読み込みデータの表示
stock_data.head()

,Open,High,Low,Volume,Adj Close,ema_12,ema_26,Open000001.SS,High000001.SS,Low000001.SS,Adj Close000001.SS,Open^AORD,High^AORD,Low^AORD,Adj Close^AORD,Open^AXJO,High^AXJO,Low^AXJO,Adj Close^AXJO,Open^BFX,High^BFX,Low^BFX,Adj Close^BFX,Open^BSESN,High^BSESN,Low^BSESN,Open^BVSP,High^BVSP,Low^BVSP,Adj Close^BVSP,Open^DJI,High^DJI,Low^DJI,Open^FCHI,High^FCHI,Low^FCHI,Adj Close^FCHI,Open^GDAXI,High^GDAXI,Low^GDAXI,Adj Close^GDAXI,Open^GSPC,High^GSPC,Low^GSPC,Adj Close^GSPC,Open^GSPTSE,High^GSPTSE,Low^GSPTSE,Adj Close^GSPTSE,Open^HSI,High^HSI,Low^HSI,Adj Close^HSI,High^IPSA,Low^IPSA,Adj Close^IPSA,Open^IXIC,High^IXIC,Low^IXIC,Adj Close^IXIC,Open^JKSE,High^JKSE,Low^JKSE,Adj Close^JKSE,Open^KS11,High^KS11,Low^KS11,Adj Close^KS11,Open^MERV,High^MERV,Low^MERV,Adj Close^MERV,Open^MXX,High^MXX,Low^MXX,Adj Close^MXX,Open^N100,High^N100,Low^N100,Adj Close^N100,Open^NYA,High^NYA,Low^NYA,Adj Close^NYA,Open^RUT,High^RUT,Low^RUT,Adj Close^RUT,High^TWII,Low^TWII,Adj Close^TWII,Open^VIX,High^VIX,Low^VIX,Open^XAX,High^XAX,Low^XAX,Adj Close^XAX,USD,GBP,EUR,CAD,CHF,SEK,DKK,NOK,AUD,NZD,ZAR,BHD,IDR(100),CNY,HKD,INR,PHP,SGD,KRW(100),THB,KWD,SAR,AED,MXN,TWD,pvema_5,pvema_12,pvema_26,answer,x0,x1,x2,x3,x4,x5,x6,x7,x0^2,x0 x1,x0 x2,x0 x3,x0 x4,x0 x5,x0 x6,x0 x7,x1^2,x1 x2,x1 x3,x1 x4,x1 x5,x1 x6,x1 x7,x2^2,x2 x3,x2 x4,x2 x5,x2 x6,x2 x7,x3^2,x3 x4,x3 x5,x3 x6,x3 x7,x4^2,x4 x5,x4 x6,x4 x7,x5^2,x5 x6,x5 x7,x6^2,x6 x7,x7^2
0,-0.006250,0.024889,-0.003604,2.643314,0.018700,0.001987,0.001856,-0.016024,-0.004302,0.004251,0.007344,-0.007255,-0.006006,-0.001020,-0.002846,-0.008053,-0.007226,-0.000710,-0.002805,-0.004067,0.006438,0.002904,0.010276,-0.001324,-0.010327,-0.009173,-0.014983,-0.014038,-0.013892,-0.012080,-0.003927,-0.001217,-0.012214,-0.011637,-0.009375,-0.009456,0.001065,-0.016036,-0.010612,-0.004488,-0.005506,-0.008530,-0.006707,-0.014114,-0.009993,-0.003407,-0.004167,-0.006936,-0.006474,-0.018231,-0.018280,-0.003118,-0.004052,-0.003805,-0.000105,0.005605,-0.014401,-0.014141,-0.018726,-0.011112,0.003838,0.000590,0.004085,-0.001906,0.023984,0.016071,0.030788,0.014635,-0.002590,-0.027515,-0.014697,-0.008481,-0.007267,-0.009166,-0.017076,-0.017050,-0.006375,-0.007674,-0.007250,-0.001550,-0.009459,-0.009459,-0.009459,-0.009459,-0.007948,-0.005887,-0.009231,-0.007772,-0.003112,-0.003727,0.008201,0.015167,0.054611,0.014644,0.001202,-0.003871,-0.007277,-0.008593,0.000000,-0.002711,-0.001877,0.003238,-0.001622,-0.003846,-0.001268,-0.000656,0.001690,-0.000849,0.009338,0.000000,-0.007246,0.000000,0.000000,0.003663,0.000000,-0.000831,-0.008893,-0.006536,0.003609,0.000000,0.000000,-0.002027,0.000000,0.012620,0.010905,0.023161,1.0,-0.031376,-0.032862,0.000000,-0.003411,0.002781,0.838616,0.003563,0.001813,0.000984,0.001031,-0.000000,0.000107,-0.000087,-0.026312,-0.000112,-0.000057,0.001080,-0.000000,0.000112,-0.000091,-0.027558,-0.000117,-0.000060,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000012,-0.000009,-0.002860,-0.000012,-0.000006,0.000008,0.002332,0.000010,5.041579e-06,0.703278,0.002988,0.001520,0.000013,6.459435e-06,3.286352e-06
1,0.026954,0.004337,0.028933,0.208818,0.000874,0.001814,0.001782,0.007755,0.024780,0.000608,0.029596,-0.002965,-0.004128,-0.003605,-0.001832,-0.002805,-0.004479,-0.003582,-0.001539,0.007597,0.008347,0.009030,0.007857,-0.010703,0.008073,0.001284,-0.014307,0.001873,-0.001531,0.021016,-0.010887,-0.007542,0.001753,-0.002981,-0.004875,-0.010073,-0.020282,-0.001732,-0.006093,-0.010315,-0.005091,-0.009993,-0.007376,0.000339,0.000835,-0.004867,-0.003182,-0.007444,-0.007370,-0.005971,-0.001443,-0.003247,-0.000239,0.006706,0.006440,0.003402,-0.018269,-0.006910,-0.000254,0.003026,0.001560,0.021175,0.003847,0.029382,0.014400,0.001065,0.011734,-0.000631,-0.004941,0.004482,-0.026619,-0.037923,-0.017218,-0.008022,-0.003079,0.008560,-0.005135,-0.005065,-0.012784,-0.016449,0.000559,0.000559,0.000559,0.000559,-0.007772,-0.005882,-0.002279,0.003564,0.000140,-0.006939,-0.013912,0.025245,-0.018564,0.003093,-0.008593,-0.008558,-0.006889,-0.004

In [4]:
# DataFrameをNumPy配列へ変換
# 正解ラベルの変換
answers = stock_data.answer.values
# 説明変数の変換
stock_data.drop(columns=['answer'], inplace=True)
explanatory_variable = stock_data.values

In [5]:
# スケール変換        
ms = MinMaxScaler()
ms.fit(explanatory_variable)
explanatory_variable = ms.transform(explanatory_variable)

In [6]:
# データの分割（データの80%を訓練用に、20％をテスト用に分割する）
X_train, X_test, y_train, y_test = train_test_split(explanatory_variable , answers, test_size=0.2, random_state=1,shuffle = False)

In [7]:
# グリッドサーチするパラメータを設定
parameters = {'C':[0.01,0.1,1,10,100],'loss':['hinge', 'squared_hinge']}
# グリッドサーチを実行
lsvc =  LinearSVC(random_state=1)
grid_search = GridSearchCV(lsvc, param_grid=parameters, cv=5)
grid_search = grid_search.fit(X_train , y_train)

# グリッドサーチ結果(最適パラメータ)を取得
GS_C, GS_loss = grid_search.best_params_.values()
print ("最適パラメータ：{}".format(grid_search.best_params_))

最適パラメータ：{'C': 0.1, 'loss': 'squared_hinge'}


In [8]:
# 最適パラメーターを指定して学習
clf = LinearSVC(loss=GS_loss, C=GS_C, random_state=1)
clf.fit(X_train , y_train)

LinearSVC(C=0.1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=1, tol=0.0001,
     verbose=0)

In [9]:
# 学習後のモデルによるテスト
# トレーニングデータを用いた予測
y_train_pred = clf.predict(X_train)
# テストデータを用いた予測
y_val_pred = clf.predict(X_test)

In [10]:
# 正解率の計算
train_score = accuracy_score(y_train, y_train_pred)
test_score = accuracy_score(y_test, y_val_pred)
# 正解率を表示
print("トレーニングデータに対する正解率：" + str(train_score * 100) + "%")
print("テストデータに対する正解率：" + str(test_score * 100) + "%")

トレーニングデータに対する正解率：65.35819430814524%
テストデータに対する正解率：61.830065359477125%


In [11]:
#  交差検証
## 5分割し交差検証
scores = cross_val_score(clf, explanatory_variable, answers, cv=5)
## 各分割におけるスコア
print('Cross-Validation scores: {}'.format(scores))
## スコアの平均値
print('Average score: {}'.format(np.mean(scores)))

Cross-Validation scores: [0.63529412 0.63529412 0.62696335 0.60994764 0.61649215]
Average score: 0.6247982753310748


In [12]:
# モデルベース特徴量選択による特徴量選択
# estimatorとしてLightGBMを使用。重要度がmedian（中央値）以上のものを選択
selector_LGBM = SelectFromModel(lgb.LGBMClassifier(random_state=1), threshold="median")
selector_LGBM.fit(X_train , y_train)

# 選択された特徴量による学習：モデルベース特徴量選択
# 選択された特徴量のみにデータを変換
X_train_LGBM = selector_LGBM.transform(X_train)
X_test_LGBM = selector_LGBM.transform(X_test)

# 学習
clf.fit(X_train_LGBM, y_train)

# 正解率の算出
y_train_pred_LGBM = clf.predict(X_train_LGBM)
y_val_pred_LGBM = clf.predict(X_test_LGBM)
train_score = accuracy_score(y_train, y_train_pred_LGBM)
test_score = accuracy_score(y_test, y_val_pred_LGBM)

print("モデルベース特徴量選択(LightGBM)：トレーニングデータに対する正解率：" + str(train_score * 100) + "%")
print("モデルベース特徴量選択(LightGBM)：テストデータに対する正解率：" + str(test_score * 100) + "%")

#  交差検証
## 選択された特徴量のみにデータを変換
explanatory_variable_LGBM = selector_LGBM.transform(explanatory_variable)
## 5分割し交差検証
scores = cross_val_score(clf, explanatory_variable_LGBM, answers, cv=5)
## 各分割におけるスコア
print('Cross-Validation scores: {}'.format(scores))
## スコアの平均値
print('Average score: {}'.format(np.mean(scores)))

モデルベース特徴量選択(LightGBM)：トレーニングデータに対する正解率：64.4095518482172%
モデルベース特徴量選択(LightGBM)：テストデータに対する正解率：62.61437908496732%
Cross-Validation scores: [0.64313725 0.64313725 0.61256545 0.61780105 0.62172775]
Average score: 0.6276737501283236


In [14]:
# モデルベース特徴量選択による特徴量選択
# estimatorとしてXGboostを使用。重要度がmedian（中央値）以上のものを選択
selector_XGB = SelectFromModel(xgb.XGBClassifier(random_state=1), threshold="median")
selector_XGB.fit(X_train , y_train)

# 選択された特徴量による学習：モデルベース特徴量選択
# 選択された特徴量のみにデータを変換
X_train_XGB = selector_XGB.transform(X_train)
X_test_XGB = selector_XGB.transform(X_test)

# 学習
clf.fit(X_train_XGB, y_train)

# 正解率の算出
y_train_pred_XGB = clf.predict(X_train_XGB)
y_val_pred_XGB = clf.predict(X_test_XGB)
train_score = accuracy_score(y_train, y_train_pred_XGB)
test_score = accuracy_score(y_test, y_val_pred_XGB)

print("モデルベース特徴量選択(XGboost)：トレーニングデータに対する正解率：" + str(train_score * 100) + "%")
print("モデルベース特徴量選択(XGboost)：テストデータに対する正解率：" + str(test_score * 100) + "%")

#  交差検証
## 選択された特徴量のみにデータを変換
explanatory_variable_XGB = selector_XGB.transform(explanatory_variable)
## 5分割し交差検証
scores = cross_val_score(clf, explanatory_variable_XGB, answers, cv=5)
## 各分割におけるスコア
print('Cross-Validation scores: {}'.format(scores))
## スコアの平均値
print('Average score: {}'.format(np.mean(scores)))

モデルベース特徴量選択(XGboost)：トレーニングデータに対する正解率：64.4095518482172%
モデルベース特徴量選択(XGboost)：テストデータに対する正解率：61.830065359477125%
Cross-Validation scores: [0.6379085  0.65490196 0.62303665 0.61387435 0.62303665]
Average score: 0.6305516202990795


In [15]:
# RFEによる特徴量選択
# estimatorとしてLightGBMを使用。上位50個の特徴量を抽出する
selector_RFE = RFE(lgb.LGBMClassifier(random_state=1), n_features_to_select=50)
selector_RFE.fit(X_train , y_train)

# 選択された特徴量のみにデータを変換
X_train_RFE = selector_RFE.transform(X_train)
X_test_RFE = selector_RFE.transform(X_test)

# 学習
clf.fit(X_train_RFE, y_train)

# 正解率の算出
y_train_pred_RFE = clf.predict(X_train_RFE)
y_val_pred_RFE = clf.predict(X_test_RFE)
train_score = accuracy_score(y_train, y_train_pred_RFE)
test_score = accuracy_score(y_test, y_val_pred_RFE)

print("RFEによる特徴量選択：トレーニングデータに対する正解率：" + str(train_score * 100) + "%")
print("RFEによる特徴量選択：テストデータに対する正解率：" + str(test_score * 100) + "%")

#  交差検証
## 選択された特徴量のみにデータを変換
explanatory_variable_RFE = selector_RFE.transform(explanatory_variable)
## 5分割し交差検証
scores = cross_val_score(clf, explanatory_variable_RFE, answers, cv=5)
## 各分割におけるスコア
print('Cross-Validation scores: {}'.format(scores))
## スコアの平均値
print('Average score: {}'.format(np.mean(scores)))

RFEによる特徴量選択：トレーニングデータに対する正解率：64.7693817468106%
RFEによる特徴量選択：テストデータに対する正解率：61.830065359477125%
Cross-Validation scores: [0.6496732  0.65228758 0.61518325 0.62303665 0.62172775]
Average score: 0.6323816856585566
